# Ceneo Scraper

## Lista kroków "manualnych"

1. Wejdź na stronę produktu / z opiniami o produkcie
2. Dla każdej opinii na stronie\
    A. Skpiuj opinię\
    B. Wklej opinię do edytora tekstu
3. Przejdz do klejnej strony z opiami
4. Powtarzaj kroki 2-3 dopóki są strony z opiniami
## Lista kroków scrapera

1. Wysłanie żądania dostępu do zasobu - strona z opiniami o produkcie
2. Wydobycie z kodu HTML fragmentów odpowiadających opiniom
3. Dla każdej opinii na stronie/
    A. Wydobycie z kodu opini jej składowych do słownika/
    B. Dodanie słownika reprezentującego pojedynczą opinię do listy
4. Przejdź do kolejnej strony z opiniami
5. Powtarzaj kroki 1-4 dopóki są strony z opiniami
6. Zapis listy słowników do pliku JSON

 ## Struktura opini w serwisie Ceneo.pl

|składowa|nazwa|selektor|
|--------|-----|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autor|author|.user-post__author-name|
|rekomendacja|recommendation|.user-post__author-recomendation > em|
|liczba gwiazdek|stars|.user-post__score-count|
|treść opinii|content|div.user-post__text|
|lista zalet|pros|.review-feature__title--positives ~ .review-feature__item|
|lista wad|cons|.review-feature__title--negaives ~ .review-feature__item|
|data wystawienia opinii|post_date| span.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date| span.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|useful|.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|useless|.vote-no > span|


.user-post[data-entry-id]

### 1. Importowanie Bibliotek

In [8]:
import requests
from bs4 import BeautifulSoup
import os
import json

### 2. Definicje funkcji

In [9]:
def get_data(ancestor, selector, attribute=None, return_list=False):
    if return_list:
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if attribute:
        if selector:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        return ancestor[attribute].strip()
    try:
        return ancestor.select_one(selector).text.strip()
    except AttributeError:
        return None

### 3. Definicje stalych i zmiennych globalnych

In [10]:
selectors = {
    'opinion_id': (None, "data-entry-id"),
    'author': ("span.user-post__author-name",),
    'recommendation': ("span.user-post__author-recomendation > em",),
    'stars': ("span.user-post__score-count",),
    'content': ("div.user-post__text",),
    'pros': ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
    'cons': ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
    'post_date': ("span.user-post__published > time:nth-child(1)","datetime"),
    'purchase_date': ("span.user-post__published > time:nth-child(2)","datetime"),
    'useful': ("button.vote-yes > span",),
    'useless': ("button.vote-no > span",),
}

### 4. Podanie adresu pierwwszej strony z opnmiami na podstawie kodu produktu

In [11]:
product_code = input("Podaj kod produktu z Ceneo.pl: ")
url = f"https://www.ceneo.pl/{product_code}#tab=reviews"

### 5. Pobranie wszystkich opini o produkcie z serwisu Ceneo.pl

In [12]:
all_opinions = []
while(url):
    print(url)
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: get_data(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://ceneo.pl"+page.select_one("a.pagination__next")["href"]
    except TypeError:
        url = None

https://www.ceneo.pl/104770639#tab=reviews
https://ceneo.pl/104770639/opinie-2
https://ceneo.pl/104770639/opinie-3
https://ceneo.pl/104770639/opinie-4
https://ceneo.pl/104770639/opinie-5


### 6. Zapis opni do plikui json

In [15]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
jf = open(f"opinions/{product_code}.json", "w", encoding="UTF-8")
json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
jf.close()

In [14]:

# all_opinions = []
# for opinion in opinions:
#     single_opinion = {
#         'opinion_id': opinion["data-entry-id"],
#         'author':  opinion.select_one(".user-post__author-name").text,
#         'recommendation': opinion.select_one(".user-post__author-recomendation > em").text,
#         'stars': opinion.select_one(".user-post__score-count").text,
#         'content': opinion.select_one(".user-post__text").text,
#         'pros': [p.text for p in opinion.select(".review-feature__title--positives ~ .review-feature__item")],
#         'cons': [c.text for c in opinion.select(".review-feature__title--negaives ~ .review-feature__item")],
#         'post_date': opinion.select_one(".user-post__published > time:nth-child(1)")['datetime'],
#         'purchase_date': opinion.select_one(".user-post__published > time:nth-child(2)")['datetime'] if  opinion.select_one(".user-post__published > time:nth-child(2)") else '',
#         'useful': opinion.select_one(".vote-yes > span").text,
#         'uselesss': opinion.select_one(".vote-no > span").text,
#     }

#     all_opinions.append(single_opinion)
#     #print( single_opinion )


# print(all_opinions)